In [ ]:
# prompt: connect to drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install transformers datasets bert-score rouge-score nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download punkt_tab data
nltk.download('wordnet') # Download wordnet data

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!pip install transformers datasets bert-score rouge-score nltk

In [ ]:
import torch
import torch.nn as nn
from transformers import T5Tokenizer, T5EncoderModel
from datasets import load_dataset
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from nltk.translate.meteor_score import single_meteor_score
from nltk.tokenize import sent_tokenize
from collections import OrderedDict

# Sentence Splitter
def split_into_sentences(text):
    return sent_tokenize(text)

# Tokenizer Function
def tokenize_sentences(sentences, tokenizer):
    return tokenizer(sentences, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# T5 Encoder-based Extractive Summarization Model
class T5ForExtractiveSummarization(nn.Module):
    def __init__(self, pretrained_model="google/flan-t5-small"):
        super().__init__()
        self.encoder = T5EncoderModel.from_pretrained(pretrained_model)
        self.classifier = nn.Linear(self.encoder.config.d_model, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # first token representation
        logits = self.classifier(cls_output).squeeze(-1)
        return logits

# Evaluation Function
def evaluate_model(model_path, pretrained_model="google/flan-t5-small", max_samples=10000):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Evaluating on device: {device}")

    # Load model
    model = T5ForExtractiveSummarization(pretrained_model).to(device)
    state_dict = torch.load(model_path, map_location=device)
    new_state_dict = OrderedDict((k[7:] if k.startswith('module.') else k, v) for k, v in state_dict.items())
    model.load_state_dict(new_state_dict)
    model.eval()

    tokenizer = T5Tokenizer.from_pretrained(pretrained_model)
    dataset = load_dataset("gigaword", split=f"validation[:{max_samples}]")

    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    meteor_total, rouge1_total, rouge2_total, rougeL_total = 0, 0, 0, 0
    references, predictions = [], []

    for example in dataset:
        article = example['document']
        summary = example['summary']
        sentences = split_into_sentences(article)

        if len(sentences) < 3:
            sentences += [""] * (3 - len(sentences))

        tokenized = tokenize_sentences(sentences, tokenizer)
        input_ids = tokenized['input_ids'].to(device)
        attention_mask = tokenized['attention_mask'].to(device)

        try:
            logits = model(input_ids, attention_mask).squeeze(0)
        except Exception:
            logits = torch.zeros(len(sentences), device=device)

        if logits.dim() == 0 or len(logits) != len(sentences):
            logits = torch.zeros(len(sentences), device=device)

        top_indices = sorted(range(len(logits)), key=lambda i: logits[i], reverse=True)[:3]
        pred_summary = " ".join([sentences[i] for i in top_indices])

        scores = rouge.score(summary, pred_summary)
        rouge1_total += scores['rouge1'].fmeasure
        rouge2_total += scores['rouge2'].fmeasure
        rougeL_total += scores['rougeL'].fmeasure
        meteor_total += single_meteor_score(summary.split(), pred_summary.split())
        references.append(summary)
        predictions.append(pred_summary)

    precision, recall, f1 = bert_score(predictions, references, lang='en', verbose=False)
    n = len(predictions)

    print(f"\n✅ Final Evaluation on {n} samples")
    print(f"ROUGE-1 F1: {rouge1_total / n:.4f}")
    print(f"ROUGE-2 F1: {rouge2_total / n:.4f}")
    print(f"ROUGE-L F1: {rougeL_total / n:.4f}")
    print(f"METEOR:     {meteor_total / n:.4f}")
    print(f"BERTScore P/R/F1: {precision.mean().item():.4f} / {recall.mean().item():.4f} / {f1.mean().item():.4f}")

    torch.cuda.empty_cache()


In [ ]:
evaluate_model("/best_flan_t5_gigaword_model.pt", pretrained_model="google/flan-t5-small", max_samples=10000)
# Pass "google/flan-t5-small" as the pretrained_model argument explicitly.

Evaluating on device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ Final Evaluation on 10000 samples
ROUGE-1 F1: 0.2995
ROUGE-2 F1: 0.1020
ROUGE-L F1: 0.2605
METEOR:     0.4160
BERTScore P/R/F1: 0.8358 / 0.9078 / 0.8701
